In [ ]:
%matplotlib inline
# Importing necessary packages:
from glob import glob
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PipeLine import *

In [ ]:
csv_files = glob("../N*-analyze_bug/N*.csv")

In [ ]:
geometry = 'cylinder'
df_name = 'all_in_one'
ext=['-properties.csv']
properties_files = PipeLine.file_reader(csv_files,extensions=ext)
properties_df = PipeLine.df_of_properties(properties_files, df_name, geometry, to_file=True,index_col=0)
properties_df = properties_df.round(4)


In [ ]:
def df_of_properties(ensemble_files, ensemble_name, geometry, to_file=True,**kwargs):
    """
    This function computes create a dataframe from the input parameters and equilibrium properties of individual ensembles.
    The differentce in ensembles comes from the different in the seed of random number generator used in Lammps velocity and fix langevin commands.
    
    Input:
    ensemble_files: a list of addresses of properties of ensmebles 
    ensemble_name (str): name of output file.
    geomtery (str): shape of the simulation box
    to_file (bool): whether save to file or not
    
    Return:
    A dataframe of the input parameters and equilibrium properties of all the ensembles.
    
    Requirements:
    Pandas, my own cellAttributes class
    """
    ensembles_list = []
    for ensemble_file in ensemble_files: #files are organized based on their name
        df = pd.read_csv(ensemble_file[0],comment='#',**kwargs)
        ensembles_list.append(df)
    df_of_ensembles = pd.concat(ensembles_list)
    df_of_ensembles.reset_index(inplace=True,drop=True)
    cell_attrs = cellAttributes(df_of_ensembles.filename[0],geometry=geometry)
    if to_file:
        output = 'N'+str(cell_attrs.nmon)+'D'+str(cell_attrs.dcyl)+'ac'+str(cell_attrs.dcrowd)+'-'+ensemble_name+'.csv'
        df_of_ensembles.to_csv(output)
    return df_of_ensembles

In [ ]:
csv_files = glob("../N*-analyze_bug/N*.csv")
geometry = 'cylinder'
df_name = 'all_in_one-ens_avg'
ext=['-properties-ens_avg.csv']
ens_evg_properties_files = PipeLine.file_reader(csv_files,extensions=ext)
ensembles_list = []
for ensemble_file in ens_evg_properties_files: #files are organized based on their name
    print(ensemble_file)
    header = ensemble_file[0].split('/')[1].split('-')[0]
    middle = ensemble_file[0].split('/')[-1].split('-')[1]
    footer = ensemble_file[0].split('/')[-1].split('-')[-1].split('.csv')[0]+'-normalized.csv'
    output = header +'-'+middle+'-'+ footer 
    print(output)
    df = pd.read_csv(ensemble_file[0],comment='#',index_col=0)
    unique_simulations = [list(input_set) for input_set in list(df.groupby(['nmon','dcyl','dcrowd']).size().index)]
    for nmon, dcyl, dcrowd in unique_simulations:
        condition = (ens_avg_properties_df['nmon'] == nmon) & (ens_avg_properties_df['dcyl'] == dcyl) & (ens_avg_properties_df['dcrowd'] == dcrowd) & (ens_avg_properties_df['vfrc_crowd'] == 0)
        fsd = ens_avg_properties_df[condition]['fsd'].values[0]
        gyr = ens_avg_properties_df[condition]['gyr'].values[0]
        rflory = ens_avg_properties_df[condition]['rflory'].values[0]
    df['fsd_normalized'] = df['fsd'] / fsd
    df['gyr_normalized'] = df['gyr'] / gyr
    df['rflory_normalized'] = df['rflory'] / rflory
    df['vfrc_c_normalized'] = (df['dmon'] * df['vfrc_crowd']) / df['dcrowd'] 
    df.to_csv(output)

In [ ]:
    
    
    
    ensembles_list.append(df)
df_of_ensembles = pd.concat(ensembles_list)
df_of_ensembles.reset_index(inplace=True,drop=True)
cell_attrs = cellAttributes(df_of_ensembles.filename[0],geometry=geometry)
if to_file:
    output = 'N'+str(cell_attrs.nmon)+'D'+str(cell_attrs.dcyl)+'ac'+str(cell_attrs.dcrowd)+'-'+ensemble_name+'.csv'
    df_of_ensembles.to_csv(output)
    
for df in ens_evg_properties_files:
    

In [ ]:
csv_files = glob("./N*.csv")
geometry = 'cylinder'
df_name = 'all_in_one-ens_avg'
ext=['-properties-ens_avg-normalized.csv']
ens_evg_properties_files = PipeLine.file_reader(csv_files,extensions=ext)
ens_avg_properties_df = PipeLine.df_of_properties(ens_evg_properties_files, df_name, geometry, to_file=True,index_col=0)
ens_avg_properties_df = ens_avg_properties_df.round(4)

In [ ]:
unique_simulations = [list(input_set) for input_set in list(ens_avg_properties_df.groupby(['nmon','dcyl','dcrowd']).size().index)]
for nmon, dcyl, dcrowd in unique_simulations:
    condition = (ens_avg_properties_df['nmon'] == nmon) & (ens_avg_properties_df['dcyl'] == dcyl) & (ens_avg_properties_df['dcrowd'] == dcrowd) & (ens_avg_properties_df['vfrc_crowd'] == 0)
    print(ens_avg_properties_df[condition]['fsd'])

In [ ]:
sns.set_context('paper')
sns.set_style("ticks")
fig, axes = plt.subplots(nrows=3,ncols=1,sharex=True)#,figsize=(16,9)
#sns.color_palette("hls", 3)
#fig, ax = plt.subplots(nrows=1,ncols=1)#,figsize=(16,9))
g1 = sns.lineplot(x="vfrc_c_normalized", y="fsd_normalized", hue='dcyl',style='dcrowd', markers=True, data=ens_avg_properties_df,ax=axes[0])
g2 = sns.lineplot(x="vfrc_c_normalized", y="gyr_normalized", hue='dcyl',style='dcrowd', markers=True, data=ens_avg_properties_df, legend=False,ax=axes[1])
g3 = sns.lineplot(x="vfrc_c_normalized", y="rflory_normalized", hue='dcyl',style='dcrowd', markers=True, data=ens_avg_properties_df,legend=False, ax=axes[2])
g1.legend(title=[r'D',r'$a_c$'],bbox_to_anchor=(1.05, 1), loc=2)
legend = ax.legend()
for t in legend.get_texts()[::2]:
    xfm = transforms.offset_copy(t.get_transform(), ax.figure, x=-10, units="points")
    t.set_transform(xfm)
g1.set_ylabel(r'$\frac{L_{FSD}(\phi_c)}{L_{FSD}(0)}$')
g2.set_ylabel(r'$\frac{R_{ROG}(\phi_c)}{R_{ROG}(0)}$')
g3.set_ylabel(r'$\frac{R_{Flory}(\phi_c)}{R_{Flory}(0)}$')
picname = "Chain_size-all_in_one"
plt.savefig(picname+'.pdf',dpi=300,bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)#,figsize=(16,9))
ax = 

In [ ]:
#%%time
#df_name = '_ens_avg_z'
#direction = 'cyl_z'
#ens_avg_rho_z = PipeLine.rho_phi_generator(ens_evg_zhist, ens_avg_properties, df_name, geometry, direction)

#df_name = '_ens_avg_theta'
#direction = 'cyl_theta'
#ens_avg_rho_theta = PipeLine.rho_phi_generator(ens_evg_thetahist, ens_avg_properties, df_name, geometry, direction)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))
fontsize = 24
color = 'tab:blue'
line1, = ax.plot(ens_avg_phi_r.iloc[:,0], markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(ens_avg_rho_r.iloc[:,0], rho, markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
import os
for direct in rhist_df.columns:
    path = os.path.join("./", direct)
    os.mkdir(path)

In [ ]:
fpath = hist_files[24]

histo_collections = np.loadtxt(fpath[0],dtype=np.int32)
bin_edges = np.around(np.loadtxt(fpath[1]),decimals=2)
cell_attrs = PipeLine.cellAttributes(fpath[0],geometry='cylinder',printname=True)
filename = cell_attrs.filename
cylindrical_shell_integrand = lambda r: 2 * np.pi * cell_attrs.dcyl * r

# the sum of rho is not equal to the bulk number density (r=infiity) natom/cell_vol
# this arises from the way we descritize the local number desnity.
rho = PipeLine.local_number_density(histo_collections, bin_edges, cylindrical_shell_integrand)

rmon = cell_attrs.dmon / 2.0
bin_centers = np.around((bin_edges[:-1] + bin_edges[1:]) / 2.0,decimals=2)

range_of_bins = PipeLine.bin_edge_index(rmon, bin_centers, bin_edges)
volume_shares = PipeLine.vol_shares(rmon, bin_centers, bin_edges, range_of_bins, PipeLine.sphere_cylinder_intersection)

# the sum of phi is not equal to the bulk volume fraction (r=infiity) natom*vol_per_atom/cell_vol
# this arises from the way we descritize the local volume fraction and the way we relate it to the local number density.
phi = PipeLine.local_volume_fraction(bin_centers, rho, volume_shares)

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))

fontsize = 24
color = 'tab:blue'
line1, = ax.plot(bin_centers, phi, label = r"$\phi_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(bin_centers, rho, label = r"$\rho_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
nens = 0
hists = 0
phi_c = 0.0 

for filename in ens_avg_properties.filename:
    condition = ens_avg_properties['filename']==filename
    for col in rhist_dfs.columns:
        if col.split('ens')[0] == filename.split('_ens_avg')[0]:
            #print(rhist_dfs[col])
            PipeLine.histo_plotter(ax, rhist_dfs[col].values, lambda r: 2 * ens_avg_properties[condition].dcyl / 2.0 * r, rhist_dfs.index.values, label = r"$\phi_c={}$".format(ens_avg_properties[condition].vfrc_crowd), markersize=0.1)
    #ens_evg_rhist
    
#PipeLine.histo_plotter(ax, hists, lambda r: 2 * cell_attrs.dcyl / 2.0 * r, edges, label = r"$\phi_c={}$".format(phi_c), markersize=0.1)
ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")